In [12]:
import pandas as pd
import numpy as np
import os
import nltk.classify.util
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.cross_validation import KFold, train_test_split
from BeautifulSoup import BeautifulSoup
from pymongo import MongoClient


In [18]:
#PyMongo variables
client = MongoClient()
db = client['reviews']
collection = db['movies']
reviews = collection.find()

#Lists for X and y
review_list = []
opinion_list = []
word_list = []

n_features = 2000
n_topics = 2
n_top_words = 20
pos_words = []
neg_words = []

for review in reviews:
    opinion_list.append(review['Opinion'])
    review_list.append(review['Review'])

vectorizer = TfidfVectorizer(max_features=n_features, stop_words='english'\
                , ngram_range=(1,3), sublinear_tf=True, norm='l2')
review_tfidf = vectorizer.fit_transform(review_list)
opinion_array = np.array(opinion_list)
feature_names = vectorizer.get_feature_names()

feats_train, feats_test, opinions_train, opinions_test = train_test_split(\
                    review_tfidf, opinion_array, test_size=.2)

nmf = NMF(n_components=n_topics,init='nndsvd')

X = []
kf = KFold(review_tfidf.shape[0], n_folds=5)

for small_index, large_index in kf:
    training_features = review_tfidf[large_index]
    test_features = review_tfidf[small_index]
    nmf.fit(training_features)
    X.append(nmf.transform(test_features).dot(nmf.components_))

train_X = np.mean(np.array(X),0)

logit = LogisticRegression()
nb = BernoulliNB()

logit.fit(train_X,opinions_train)
nb.fit(train_X,opinions_train)

print logit.score(feats_test,opinions_test)
print nb.score(feats_test,opinions_test)

0.425
0.505
